<a href="https://colab.research.google.com/github/Zvonok13/Trash_sort/blob/main/Servo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np
from tensorflow import keras
import serial
import asyncio

# Путь к модели
model_path = "C:/Users/zvono/PycharmProjects/od arduino/garbage_recognition_model_au70.h5"
model = keras.models.load_model(model_path)

# Подключение к порту Arduinoqq
ser = serial.Serial('COM3', 9600)  # Указать правильный порт и скорость

# Функция для предварительной обработки изображения
def preprocess_image(image):
    # Ресайз изображения и нормализация значений пикселей
    processed_image = cv2.resize(image, (224, 224)) / 255.0
    processed_image = np.expand_dims(processed_image, axis=0)
    return processed_image

# Функция для классификации объектов
def classify_object(image):
    # Предварительная обработка изображения
    processed_image = preprocess_image(image)

    # Классификация объектов с использованием модели
    predictions = model.predict(processed_image)
    class_index = np.argmax(predictions)
    class_label = get_class_label(class_index)

    # Отображение результата на изображении
    cv2.putText(image, f"Class: {class_label}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

    return image, class_index

# Функция для получения метки класса по индексу
def get_class_label(index):
    # Ваши метки классов
    class_labels = ["plastic", "glass", "paper", "metal"]
    return class_labels[index]

async def read_from_arduino():
    while True:
        if ser.in_waiting > 0:
            data = ser.readline().decode().strip()
            print(f"Arduino: {data}")
            if data == "1":
                return

async def send_to_arduino(class_index):
    class_label = 0
    if class_index == 0:  # plastic
        class_label = 1
    elif class_index == 1:  # glass или metal
        class_label = 2
    elif class_index == 2:  # paper
        class_label = 3
    ser.write(str(class_label).encode())

# Запуск видеопотока с камеры
async def main():
    cap = cv2.VideoCapture(0)
    while True:
        await read_from_arduino()
        ret, frame = cap.read()
        if not ret:
            break
        _, class_index = classify_object(frame)
        await send_to_arduino(class_index)
        cv2.imshow('Classified Frame', frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

# Запуск основной функции асинхронно
asyncio.run(main())
